In [ ]:
from google.colab import drive

# Google ドライブをマウント
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **国会議事録取得**

<参考・引用>

* 国会会議録検索システムAPIのご紹介　https://note.com/sanskruthiya/n/n9e8b77067b88

In [ ]:
# Python
import requests
import time
import sys
import csv
import re

base_url = "https://kokkai.ndl.go.jp/api/speech"

# URLパラメータ用の辞書を空の状態で用意し、後から順次格納する。01はヒット総数の確認用、02はデータ取得用。
params_01 = {}
params_02 = {}

# パラメータを対話的に入力する
text_input = input('検索する文字列を入力（Enterキーでスキップ） >> ')
if text_input != "":
    params_01['any'] = str(text_input)
    params_02['any'] = str(text_input)
else:
    pass

house_input = input('検索する院名を入力（Enterキーでスキップ） >> ')
if house_input != "":
    params_01['nameOfHouse'] = str(house_input)
    params_02['nameOfHouse'] = str(house_input)
else:
    pass

meeting_input = input('検索する会議名を入力（Enterキーでスキップ） >> ')
if meeting_input != "":
    params_01['nameOfMeeting'] = str(meeting_input)
    params_02['nameOfMeeting'] = str(meeting_input)
else:
    pass

speaker_input = input('検索する発言者名を入力（Enterキーでスキップ） >> ')
if speaker_input != "":
    params_01['speaker'] = str(speaker_input)
    params_02['speaker'] = str(speaker_input)
else:
    pass

from_input = input('開始日を入力 (e.g. 2020-09-01) >> ')
if re.match(r'[0-9]{4}-[0-1][0-9]-[0-3][0-9]', from_input): # 正規表現によるパターンマッチングにて入力値が有効か判定
    params_01['from'] = str(from_input)
    params_02['from'] = str(from_input)
else:
    params_01['from'] = "2020-09-01"
    params_02['from'] = "2020-09-01"
    print("'From' date is set to 2020-09-01 due to invalid input")

until_input = input('終了日を入力 (e.g. 2020-11-30) >> ')
if re.match(r'[0-9]{4}-[0-1][0-9]-[0-3][0-9]', until_input): # 正規表現によるパターンマッチングにて入力値が有効か判定
    params_01['until'] = str(until_input)
    params_02['until'] = str(until_input)
else:
    params_01['until'] = "2020-09-30"
    params_02['until'] = "2020-09-30"
    print("'Until' date is set to 2020-09-30 due to invalid input")

# ヒット件数確認用のパラメータを設定
params_01['maximumRecords'] = 1
params_01['recordPacking'] = "json"

response_01 = requests.get(base_url, params_01) # URLをエンコードしてAPIへリクエスト
jsonData_01 = response_01.json() # APIからのレスポンスをJSON形式で取得

# レスポンスに含まれているヒット件数を確認（レスポンスのJSONにレコード数の項目がない場合はクエリに問題ありと判断しエラー終了）
try:
    total_num = jsonData_01["numberOfRecords"]
except:
    print("クエリエラーにより取得できませんでした。")
    sys.exit()

# 件数を表示し、データ取得を続行するか確認
next_input = input("検索結果は " + str(total_num) + "件です。\nキャンセルする場合は 1 を、データを取得するにはEnterキーまたはその他を押してください。 >> ")
if next_input == "1":
    print('プログラムをキャンセルしました。')
    sys.exit()
else:
    pass

max_return = 100 # 発言内容は一回のリクエストにつき100件まで取得可能なため、その上限値を取得件数として設定
pages = (int(total_num) // int(max_return)) + 1 # ヒットした全件を取得するために何回リクエストを繰り返すか算定

# 全件取得用のパラメータを設定
params_02['maximumRecords'] = max_return
params_02['recordPacking'] = "json"

Records = [] # 取得データを格納するための空リストを用意

# 全件取得するためのループ処理
i = 0
while i < pages:
    i_startRecord = 1 + (i * int(max_return))
    params_02['startRecord'] = i_startRecord
    response_02 = requests.get(base_url, params_02)
    jsonData_02 = response_02.json()
    #JSONデータ内の各発言データから必要項目を指定してリストに格納する
    for list in jsonData_02['speechRecord']:
        list_kind = list['imageKind']
        list_house = list['nameOfHouse']
        list_topic = list['nameOfMeeting']
        list_date = list['date']
        list_order = list['speechOrder']
        list_speaker = list['speaker']
        list_group = list['speakerGroup']
        list_position = list['speakerPosition']
        list_speech = list['speech'].replace('\r\n', '').replace('\u3000', '') # 発言内容の文中には改行コードが含まれるため、これを半角スペースに置換
        Records.append([list_kind, list_house, list_topic, list_date, list_order, list_speaker, list_group, list_position, list_speech])

    sys.stdout.write("\r%d/%d is done." % (i+1, pages)) # 進捗状況を表示する
    i += 1
    time.sleep(0.5) # リクエスト１回ごとに若干時間をあけてAPI側への負荷を軽減する

# CSVへの書き出し
with open("kokkai_speech_" + str(total_num) + ".csv", 'w', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC) #CSVの書き出し方式を適宜指定
    csvwriter.writerow(['種別', '院名', '会議名', '日付', '発言番号', '発言者名', '発言者所属会派', '発言者肩書き', '発言内容'])
    for record in Records:
        csvwriter.writerow(record)

検索する文字列を入力（Enterキーでスキップ） >> 
検索する院名を入力（Enterキーでスキップ） >> 
検索する会議名を入力（Enterキーでスキップ） >> 本会議
検索する発言者名を入力（Enterキーでスキップ） >> 
開始日を入力 (e.g. 2020-09-01) >> 2018-01-01
終了日を入力 (e.g. 2020-11-30) >> 2023-12-31
検索結果は 16537件です。
キャンセルする場合は 1 を、データを取得するにはEnterキーまたはその他を押してください。 >> 
166/166 is done.

---
---

# 〇取得したデータのヘッダーを表示

In [ ]:
# csvファイルのパス
csv_path = '/content/kokkai_speech_16537.csv'

# csvファイルを開く
with open(csv_path, 'r', encoding='utf-8') as f:
  # csvファイルのヘッダーを取得する
  reader = csv.reader(f)
  row_name = next(reader) # ヘッダー

print("ヘッダー : ",row_name)

ヘッダー :  ['種別', '院名', '会議名', '日付', '発言番号', '発言者名', '発言者所属会派', '発言者肩書き', '発言内容']


# 〇発言内容を表示

In [ ]:
import pandas as pd

talks = pd.read_csv(csv_path)

talks['発言内容']

0        令和五年六月二十一日（水曜日）―――――――――――――令和五年六月二十一日午後一時本会議―...
1                         ○議長（細田博之君）これより会議を開きます。――――◇―――――
2        ○佐々木紀君請願上程に関する緊急動議を提出いたします。本日委員会の審査を終了した裁判所の人的...
3              ○議長（細田博之君）佐々木紀君の動議に御異議ありませんか。〔「異議なし」と呼ぶ者あり〕
4        ○議長（細田博之君）御異議なしと認めます。―――――――――――――裁判所の人的・物的充実に...
                               ...                        
16532    ○国務大臣（麻生太郎君）平成三十年度予算及び税制改正並びに平成二十九年度補正予算の御審議に当...
16533                 ○議長（伊達忠一君）国務大臣茂木敏充君。〔国務大臣茂木敏充君登壇、拍手〕
16534    ○国務大臣（茂木敏充君）経済財政政策担当大臣として、我が国経済の課題と政策運営の基本的考え方...
16535    ○議長（伊達忠一君）ただいまの演説に対する質疑は次会に譲りたいと存じますが、御異議ございませ...
16536       ○議長（伊達忠一君）御異議ないと認めます。本日はこれにて散会いたします。午後四時五十九分散会
Name: 発言内容, Length: 16537, dtype: object

# 〇発言内容を1文ずつに分解

In [ ]:
talk_list = []

for i,talk in enumerate(talks['発言内容']):
  talk = re.sub(r"○.+（.+君）"," ",talk)
  #talk = re.sub(r"〔.+〕"," ",talk)
  #talk = re.sub(r"○|—|◇|━|─|・"," ",talk)
  #talk = re.sub(r"━"," ",talk)
  #talk = re.sub(r"\s+", "", talk).strip()
  talk_tmp = re.findall(r"(?:^|。)([^。]*)(?=。)", talk, re.DOTALL)
  talk_list.append(talk_tmp)

talks['発言内容'] = talk_list

In [ ]:
talks

,種別,院名,会議名,日付,発言番号,発言者名,発言者所属会派,発言者肩書き,発言内容
0,会議録,衆議院,本会議,2023-06-21,0,会議録情報,NaN,NaN,[]
1,会議録,衆議院,本会議,2023-06-21,1,細田博之,無所属,議長,[ これより会議を開きます]
2,会議録,衆議院,本会議,2023-06-21,2,佐々木紀,自由民主党・無所属の会,NaN,"[○佐々木紀君請願上程に関する緊急動議を提出いたします, 本日委員会の審査を終了した裁判所の..."
3,会議録,衆議院,本会議,2023-06-21,3,細田博之,無所属,議長,[ 佐々木紀君の動議に御異議ありませんか]
4,会議録,衆議院,本会議,2023-06-21,4,細田博之,無所属,議長,[ 御異議なしと認めます]
...,...,...,...,...,...,...,...,...,...
16532,会議録,参議院,本会議,2018-01-22,10,麻生太郎,自由民主党,財務大臣・内閣府特命担当大臣（金融）,[ 平成三十年度予算及び税制改正並びに平成二十九年度補正予算の御審議に当たり、財政政策等の基...
16533,会議録,参議院,本会議,2018-01-22,11,伊達忠一,各派に属しない議員,議長,[ 国務大臣茂木敏充君]
16534,会議録,参議院,本会議,2018-01-22,12,茂木敏充,自由民主党,内閣府特命担当大臣（経済財政政策）,[ 経済財政政策担当大臣として、我が国経済の課題と政策運営の基本的考え方について所信を申し述...
16535,会議録,参議院,本会議,2018-01-22,13,伊達忠一,各派に属しない議員,議長,[ ただいまの演説に対する質疑は次会に譲りたいと存じますが、御異議ございませんか]


# 発言者と発言

In [ ]:
import pandas as pd

df_list = []

# 各発言者ごとにデータフレームを作成しリストに追加
for i, speaker in enumerate(talks["発言者名"]):
    df = pd.DataFrame({'日付':talks["日付"][i], '発言者名': speaker, '発言内容': talk_list[i], '院名':talks["院名"][i], '会派':talks["発言者所属会派"][i]})
    df_list.append(df)

# データフレームを整形
speaker_talk = pd.concat(df_list, ignore_index=True)

# 結果を表示
print(speaker_talk)

                日付  発言者名                                               発言内容  \
0       2023-06-21  細田博之                                        これより会議を開きます   
1       2023-06-21  佐々木紀                         ○佐々木紀君請願上程に関する緊急動議を提出いたします   
2       2023-06-21  佐々木紀  本日委員会の審査を終了した裁判所の人的・物的充実に関する請願外四百五十三請願を一括議題とし、...   
3       2023-06-21  細田博之                                 佐々木紀君の動議に御異議ありませんか   
4       2023-06-21  細田博之                                         御異議なしと認めます   
...            ...   ...                                                ...   
228709  2018-01-22  茂木敏充                         こうした取組が日本経済再生の鍵を握ると確信しています   
228710  2018-01-22  茂木敏充                   国民の皆様、議員各位の御理解と御協力をよろしくお願い申し上げます   
228711  2018-01-22  伊達忠一             ただいまの演説に対する質疑は次会に譲りたいと存じますが、御異議ございませんか   
228712  2018-01-22  伊達忠一                                         御異議ないと認めます   
228713  2018-01-22  伊達忠一                                     本日はこれにて散会いたします   

         院名           会派  
0       衆議院          無所属

In [ ]:
speaker_talk

,日付,発言者名,発言内容,院名,会派
0,2023-06-21,細田博之,これより会議を開きます,衆議院,無所属
1,2023-06-21,佐々木紀,○佐々木紀君請願上程に関する緊急動議を提出いたします,衆議院,自由民主党・無所属の会
2,2023-06-21,佐々木紀,本日委員会の審査を終了した裁判所の人的・物的充実に関する請願外四百五十三請願を一括議題とし、...,衆議院,自由民主党・無所属の会
3,2023-06-21,細田博之,佐々木紀君の動議に御異議ありませんか,衆議院,無所属
4,2023-06-21,細田博之,御異議なしと認めます,衆議院,無所属
...,...,...,...,...,...
228709,2018-01-22,茂木敏充,こうした取組が日本経済再生の鍵を握ると確信しています,参議院,自由民主党
228710,2018-01-22,茂木敏充,国民の皆様、議員各位の御理解と御協力をよろしくお願い申し上げます,参議院,自由民主党
228711,2018-01-22,伊達忠一,ただいまの演説に対する質疑は次会に譲りたいと存じますが、御異議ございませんか,参議院,各派に属しない議員
228712,2018-01-22,伊達忠一,御異議ないと認めます,参議院,各派に属しない議員


# データフレームを保存

In [ ]:
# CSVファイルに保存
speaker_talk.to_csv('/content/drive/MyDrive/JP minutes/short/JP 2018-2023', index=False)

---
---
---
#**発言等のdfと男女ラベルを結合**

In [ ]:
# csvファイルを開く
with open('/content/drive/MyDrive/JPnames 2018-2023.csv', 'r', encoding='utf-8') as f:
  # csvファイルのヘッダーを表示
  reader = csv.reader(f)
  row_name = next(reader)

names2018_2023 = pd.read_csv('/content/drive/MyDrive/JPnames 2018-2023.csv')

names2018_2023

,発言者名,"性別(男1,女0)"
0,細田博之,1
1,佐々木紀,1
2,尾辻秀久,1
3,松本剛明,1
4,舞立昇治,1
...,...,...
684,奥野総一郎,1
685,金子万寿夫,1
686,井上義久,1
687,下地幹郎,1


In [ ]:
df2 = pd.merge(speaker_talk, names2018_2023, left_on='発言者名', right_on='発言者名', how='inner')
print(df2)

                日付  発言者名                                               発言内容  \
0       2023-06-21  細田博之                                        これより会議を開きます   
1       2023-06-21  細田博之                                 佐々木紀君の動議に御異議ありませんか   
2       2023-06-21  細田博之                                         御異議なしと認めます   
3       2023-06-21  細田博之             裁判所の人的・物的充実に関する請願外四百五十三請願を一括して議題といたします   
4       2023-06-21  細田博之                      各請願は委員長の報告を省略して採択するに御異議ありませんか   
...            ...   ...                                                ...   
228709  2018-01-25  吉田博美  そこで、これから憲法改正の議論をどのように進めていかれるのか、総理のお考えを改めてお聞かせください   
228710  2018-01-25  吉田博美              来春には、天皇陛下が御退位をされ、皇太子殿下が御即位をされることになります   
228711  2018-01-25  吉田博美  憲政史上初めての事柄であり、我々国民は、これを祝福するとともに、改めて、我が国の歴史とともに...   
228712  2018-01-25  吉田博美  そのためには、政府には、天皇陛下の御退位と皇太子殿下の御即位がつつがなく行われるよう、多岐に...   
228713  2018-01-25  吉田博美  天皇陛下の御退位と皇太子殿下の御即位に向けた取組について総理にお伺いして、私の代表質問といたします   

         院名         会派  性別(男1,女0)  
0       衆議院    

In [ ]:
# CSVファイルに保存
df2.to_csv('/content/drive/MyDrive/JP minutes/short/JP 2018-2023.csv ', index=False)

---
---
---

---
---
---
#**名前だけを抽出して男女ラベル付けの準備**

In [ ]:
# "発言者名"列の重複を削除し、新しいデータフレームを作成
name = talks["発言者名"].drop_duplicates()
name2018_2023 = name.reset_index(drop=True)

# unique_speakers_dfを使って新しいリストを作成する場合
#unique_speakers_list = unique_speakers_df.tolist()

name2018_2023

0      会議録情報
1       細田博之
2       佐々木紀
3       尾辻秀久
4       松本剛明
       ...  
685    奥野総一郎
686    金子万寿夫
687     井上義久
688     下地幹郎
689     吉田博美
Name: 発言者名, Length: 690, dtype: object

In [ ]:
# CSVファイルに保存
name2018_2023.to_csv('/content/drive/MyDrive/JPnames 2018-2023.csv', index=False)

In [ ]:
# csvファイルを開く
with open('/content/drive/MyDrive/JPnames2018.csv', 'r', encoding='utf-8') as f:
  # csvファイルのヘッダーを表示
  reader = csv.reader(f)
  row_name = next(reader)

names2018 = pd.read_csv('/content/drive/MyDrive/JPnames2018.csv')

names2018

,発言者名,"性別(男1,女0)"
0,大島理森,1
1,星野剛士,1
2,冨岡勉,1
3,伊達忠一,1
4,中曽根弘文,1
...,...,...
328,福山哲郎,1
329,井上義久,1
330,下地幹郎,1
331,吉田博美,1


In [ ]:
df3 = pd.merge(name2018_2023, names2018, left_on='発言者名', right_on='発言者名', how='outer')
print(df3)

      発言者名  性別(男1,女0)
0    会議録情報        NaN
1     細田博之        NaN
2     佐々木紀        NaN
3     尾辻秀久        NaN
4     松本剛明        NaN
..     ...        ...
685  奥野総一郎        1.0
686  金子万寿夫        1.0
687   井上義久        1.0
688   下地幹郎        1.0
689   吉田博美        1.0

[690 rows x 2 columns]


In [ ]:
df3.to_csv('/content/drive/MyDrive/JPnames 2018-2023.csv', index=False)

---
---
---

---
---
---
#**2018-2023の男女比**

In [ ]:
# csvファイルを開く
with open('/content/drive/MyDrive/JP minutes/short/JP 2018-2023.csv ', 'r', encoding='utf-8') as f:
  # csvファイルのヘッダーを表示
  reader = csv.reader(f)
  row_name = next(reader)

names2018_2023df = pd.read_csv('/content/drive/MyDrive/JP minutes/short/JP 2018-2023.csv ')

names2018_2023df

,日付,発言者名,発言内容,院名,会派,"性別(男1,女0)"
0,2023-06-21,細田博之,これより会議を開きます,衆議院,無所属,1
1,2023-06-21,細田博之,佐々木紀君の動議に御異議ありませんか,衆議院,無所属,1
2,2023-06-21,細田博之,御異議なしと認めます,衆議院,無所属,1
3,2023-06-21,細田博之,裁判所の人的・物的充実に関する請願外四百五十三請願を一括して議題といたします,衆議院,無所属,1
4,2023-06-21,細田博之,各請願は委員長の報告を省略して採択するに御異議ありませんか,衆議院,無所属,1
...,...,...,...,...,...,...
228709,2018-01-25,吉田博美,そこで、これから憲法改正の議論をどのように進めていかれるのか、総理のお考えを改めてお聞かせください,参議院,自由民主党・こころ,1
228710,2018-01-25,吉田博美,来春には、天皇陛下が御退位をされ、皇太子殿下が御即位をされることになります,参議院,自由民主党・こころ,1
228711,2018-01-25,吉田博美,憲政史上初めての事柄であり、我々国民は、これを祝福するとともに、改めて、我が国の歴史とともに...,参議院,自由民主党・こころ,1
228712,2018-01-25,吉田博美,そのためには、政府には、天皇陛下の御退位と皇太子殿下の御即位がつつがなく行われるよう、多岐に...,参議院,自由民主党・こころ,1


In [ ]:
# 性別列の値ごとにカウント
gender_counts0 = names2018_2023['性別(男1,女0)'].value_counts()
gender_counts1 = names2018_2023df['性別(男1,女0)'].value_counts()

# 発言数の男女比
print(f"--人数の男女比--\n　男性： {gender_counts0[1]} 人\n {gender_counts0[1]/len(names2018_2023)} \n　女性： {gender_counts0[0]} 人\n {gender_counts0[0]/len(names2018_2023)}\n　")

# 発言数の男女比
print(f"--発言数の男女比--\n　男性： {gender_counts1[1]} 回\n {gender_counts1[1]/len(names2018_2023df)} \n　女性： {gender_counts1[0]} 回\n {gender_counts1[0]/len(names2018_2023df)}\n　")

--人数の男女比--
　男性： 577 人
 0.8374455732946299 
　女性： 112 人
 0.1625544267053701
　
--発言数の男女比--
　男性： 193374 回
 0.8454838794302054 
　女性： 35340 回
 0.15451612056979458
　
